In [1]:
# this folder is used to explore quantities
import re
import pandas as pd
import numpy as np


# cookbook = pd.read_csv('../data/all_ingredients_mapped_to_products_original.csv')
cookbook = pd.read_excel('../data/all_ingredients_mapped_to_products_hand-edited_final.xlsx')
tj_inventory = pd.read_csv("../data/trader_joes_products_v3.csv")

In [2]:
tj_inventory['unit'].unique()

array(['/1 Each', '/19 Oz', '/16 Oz', '/2 Lb', '/1 Lb', '/12 Oz', '/3 Lb',
       '/8 Oz', '/20 Oz', '/6 Oz', '/3.53 Oz', '/10 Oz', '/4 Oz',
       '/14.5 Oz', '/7 Oz', '/18 Oz', '/24 Oz', '/5 Oz', '/9.5 Oz',
       '/14 Oz', '/11 Oz', '/9 Oz', '/17.63 Oz', '/20.4 Oz', '/3 Oz',
       '/21.2 Oz', '/16.75 Oz', '/15.2 Oz', '/22 Oz', '/5.9 Oz',
       '/14.3 Oz', '/11.5 Oz', '/6.2 Oz', '/9.45 Oz', '/10.8 Oz',
       '/11.08 Oz', '/13 Oz', '/8.47 Oz', '/9.8 Oz', '/6.78 Oz',
       '/13.05 Oz', '/10.58 Oz', '/6.35 Oz', '/15.25 Oz', '/11.64 Oz',
       '/7.76 Oz', '/8.46 Oz', '/27 Oz', '/27.161 Oz', '/3.6 Oz',
       '/9.9 Oz', '/8.8 Oz', '/5.29 Oz', '/8.5 Oz', '/0.5 Lb', '/17.6 Oz',
       '/17.9 Oz', '/7.5 Oz', '/16 Fl Oz', '/64 Fl Oz', '/15 Oz',
       '/32 Fl Oz', '/5.3 Oz', '/6.5 Oz', '/32 Oz', '/9.3 Oz', '/1 Doz',
       '/9.88 Oz', '/15.5 Oz', '/10.1 Fl Oz', '/12.3 Oz', '/24.3 Oz',
       '/25 Oz', '/18.34 Oz', '/9.87 Oz', '/1 Oz', '/5 Fl Oz', '/6.7 Oz',
       '/18.25 Oz', '/10 Fl Oz

In [30]:
cookbook = cookbook.dropna(subset=['matched_products'])

In [31]:
cookbook.shape

(3110, 24)

In [34]:
# grabbing amount and units from cookbook to make
ingre_units= cookbook[['original_text','amount', 'unit']].copy()
# dropping NaNs in units for now
#ingre_units.dropna()
ingre_units['unit'].unique()

array(['tablespoon', 'cup', nan, 'bag', 'sticks', 'package', 'teaspoon',
       'blocks', 'pint', 'loaf', 'cloves', 'block', 'head', 'pound',
       'box', 'pinch', 'wedge', 'slices', 'tub', 'jar',
       'heaping teaspoons', 'bulb', 'can', 'handful', 'bottle', 'sprigs',
       'container', 'carton', 'packages', 'wheel', 'tubs', 'quart',
       'stick', 'Cloves', 'shakes', 'cans', 'Handful', 'gallon', 'ears',
       'tube', 'scoops', 'ounce', 'large scoops', 'couple sprig', 'bar',
       'microwavable bag', 'jars', 'slice', 'of', 'sprig', 'Bulb',
       'oz tubs', 'Clove', 'Pinch', 'pieces', 'shake', 'Tablespoons',
       'Tablespoon', 'oz cans', 'Tbsps', 'oz container', 'clove',
       'oz jars', 'oz bottles', 'large clove', 'sheets',
       'picoinch * teaspoon', 'handfuls', 'Tbsp', 'strips', 'half gallon',
       'medium cloves', 'Bag', 'Dash', 'stalks', 'large head', 'heads',
       'large handfuls', 'Generous handful', 'containers', 'leaf',
       'links', 'thin rounds', 'thick sl

In [38]:
ingre_units['unit'].value_counts().rename_axis('unit').reset_index(name='count').head(35)

,unit,count
0,cup,606
1,tablespoon,310
2,teaspoon,276
3,package,180
4,bag,127
5,slices,50
6,packages,50
7,can,41
8,box,34
9,container,29


In [43]:
def view_unit(unit):
    return ingre_units[ingre_units['unit'] == unit][['original_text','unit', 'amount']]

view_unit("packages")

,original_text,unit,amount
246,"1/2 package TJ’s Spicy Chicken Nuggets, sliced...",packages,0.50
258,"½ package TJ’s Uncured Apple Smoked Bacon, cho...",packages,0.50
336,1/2 package (6 ounces) TJ's Shredded Sharp Che...,packages,0.50
376,"½ package TJ’s Capellini pasta, cook according...",packages,0.50
568,"½ package TJ’s Frozen Beef Bulgogi, heated acc...",packages,0.50
781,¾ package TJ’s Organic Persian Cucumbers,packages,0.75
800,"2 packages TJ’s Chevre Fine Herbs Goat Cheese,...",packages,2.00
932,2 packages TJ’s Organic Crescent Rolls,packages,2.00
993,"2 packages TJ’s Speculoos Cookies, divided",packages,2.00
1206,"½ package TJ’s Trimmed Leeks, ends trimmed, qu...",packages,0.50


In [46]:
import sys
from pathlib import Path

# Notebook is in: project_root/quantity_exploration/
PROJECT_ROOT = Path.cwd().parents[0]

sys.path.append(str(PROJECT_ROOT))
PROJECT_ROOT

WindowsPath('C:/Users/broth/Documents/GitHub/team-no-food-waste-for-you')

In [47]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from database.config import DATABASE_URL
from database.tables import Ingredient, TJInventory, Recipe
DATABASE_URL = "sqlite:///../database/team-no-food-waste-for-you.sqlite"

engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()


In [48]:
total_ingredients = session.query(Ingredient).count()
total_ingredients

3701

In [49]:
with_matched_product = (
    session.query(Ingredient)
    .filter(Ingredient.matched_product_id.isnot(None))
    .count()
)

with_matched_product

3167

In [50]:
with_pantry_amount = (
    session.query(Ingredient)
    .filter(Ingredient.pantry_amount.isnot(None))
    .count()
)

with_pantry_amount

3072

In [51]:
with_pantry_unit = (
    session.query(Ingredient)
    .filter(Ingredient.pantry_unit.isnot(None))
    .count()
)

with_pantry_unit

2550

In [24]:
ingre_units['unit'].unique()

array(['tablespoon', 'cup', nan, 'bag', 'sticks', 'package', 'teaspoon',
       'blocks', 'pint', 'loaf', 'cloves', 'block', 'head', 'handful',
       'pound', 'box', 'pinch', 'can', 'wedge', 'slices', 'tub', 'jar',
       'heaping teaspoons', 'bulb', 'bottle', 'sprigs', 'container',
       'carton', 'packages', 'wheel', 'tubs', 'quart', 'small sprigs',
       'stick', 'Cloves', 'shakes', 'ounce', 'cans', 'scoops', 'Handful',
       'gallon', 'ears', 'tube', 'small sprig', 'large scoops',
       'couple sprig', 'bar', 'microwavable bag', 'single-serve packets',
       'stalks', 'jars', 'inch', 'slice', 'of', 'parts', 'sprig',
       'big handful', 'Bulb', 'oz tubs', 'Clove', 'Pinch', 'pieces',
       'shake', 'Tablespoons', 'Tablespoon', 'oz cans', 'Tbsps',
       'oz container', 'clove', 'oz jars', 'oz bottles', 'large clove',
       'sheets', 'picoinch * teaspoon', 'containers', 'handfuls',
       'Generous pinch', 'Tbsp', 'batches', 'strips', 'half gallon',
       'generous cup', 'R

In [4]:
start_here = [
    "cup",
    "cups",
    "tablespoon",
    "tablespoons",
    "teaspoon",
    "teaspoons",
    "ounce",
    "ounces"]
cup_table_tea_oz = ingre_units[ingre_units['unit'].isin(start_here)].copy()
cup_table_tea_oz

,amount,unit
0,4.00,tablespoon
1,2.00,tablespoon
2,1.00,tablespoon
3,1.00,cup
4,2.00,cup
...,...,...
3688,0.50,teaspoon
3693,2.00,tablespoon
3694,0.50,teaspoon
3699,0.25,teaspoon


In [5]:
tj_inventory.head()

,category,sub_category,product_name,price,unit,url
0,Fresh Fruits & Veggies,Fruits,Sugar Sweet Melon,$3.49,/1 Each,https://www.traderjoes.com/home/products/pdp/s...
1,Fresh Fruits & Veggies,Fruits,Saturn Peaches,$3.99,/19 Oz,https://www.traderjoes.com/home/products/pdp/s...
2,Fresh Fruits & Veggies,Fruits,Fruitful Medley,$3.99,/16 Oz,https://www.traderjoes.com/home/products/pdp/f...
3,Fresh Fruits & Veggies,Fruits,Seedless Lemons,$2.99,/2 Lb,https://www.traderjoes.com/home/products/pdp/s...
4,Fresh Fruits & Veggies,Fruits,Hass Avocado,$1.29,/1 Each,https://www.traderjoes.com/home/products/pdp/h...


In [6]:
tj_inventory["unit"] = tj_inventory["unit"].str.strip().str.replace("/", "", regex=False)
unit_split = tj_inventory["unit"].str.extract(r"(?P<store_amount>[\d\.]+)\s*(?P<store_unit>[A-Za-z\s]+)")
tj_inventory = pd.concat([tj_inventory, unit_split], axis=1)
tj_inventory.head()

,category,sub_category,product_name,price,unit,url,store_amount,store_unit
0,Fresh Fruits & Veggies,Fruits,Sugar Sweet Melon,$3.49,1 Each,https://www.traderjoes.com/home/products/pdp/s...,1,Each
1,Fresh Fruits & Veggies,Fruits,Saturn Peaches,$3.99,19 Oz,https://www.traderjoes.com/home/products/pdp/s...,19,Oz
2,Fresh Fruits & Veggies,Fruits,Fruitful Medley,$3.99,16 Oz,https://www.traderjoes.com/home/products/pdp/f...,16,Oz
3,Fresh Fruits & Veggies,Fruits,Seedless Lemons,$2.99,2 Lb,https://www.traderjoes.com/home/products/pdp/s...,2,Lb
4,Fresh Fruits & Veggies,Fruits,Hass Avocado,$1.29,1 Each,https://www.traderjoes.com/home/products/pdp/h...,1,Each


In [7]:
tj_inventory.columns

Index(['category', 'sub_category', 'product_name', 'price', 'unit', 'url',
       'store_amount', 'store_unit'],
      dtype='object')

In [8]:
tj_units = tj_inventory[['store_amount', 'store_unit']]
# tj_units
tj_units['store_unit'].unique()

array(['Each', 'Oz', 'Lb', 'Fl Oz', 'Doz', 'Pint', 'Qt'], dtype=object)

In [9]:
# focusing just on Oz for now
tj_oz = tj_inventory[tj_inventory["store_unit"].str.lower() == "oz"].copy()
tj_oz.head()

,category,sub_category,product_name,price,unit,url,store_amount,store_unit
1,Fresh Fruits & Veggies,Fruits,Saturn Peaches,$3.99,19 Oz,https://www.traderjoes.com/home/products/pdp/s...,19,Oz
2,Fresh Fruits & Veggies,Fruits,Fruitful Medley,$3.99,16 Oz,https://www.traderjoes.com/home/products/pdp/f...,16,Oz
9,Fresh Fruits & Veggies,Fruits,Cherry Tomatoes on the Vine,$4.99,12 Oz,https://www.traderjoes.com/home/products/pdp/c...,12,Oz
15,Fresh Fruits & Veggies,Fruits,Organic Cranberries,$3.29,12 Oz,https://www.traderjoes.com/home/products/pdp/o...,12,Oz
21,Fresh Fruits & Veggies,Fruits,Organic Strawberries 1 Lb,$6.99,16 Oz,https://www.traderjoes.com/home/products/pdp/o...,16,Oz


In [10]:
# first since most of ingre_units and tj_units are cups, tablespoons, and oz we will focus on those to see what this would look like

unit_to_oz = {
    "cup": 8,
    "cups": 8,
    "tablespoon": 0.5,
    "tablespoons": 0.5,
    "teaspoon": 0.1667,
    "teaspoons": 0.1667,
    "ounce": 1,
    "ounces": 1
}

def convert_to_oz(amount, unit):
    """
    Converts a recipe's quantity + unit to ounces.
    """
    if pd.isna(unit) or unit.lower() not in unit_to_oz:
        return None
    try:
        return float(amount) * unit_to_oz[unit]
    except (TypeError, ValueError):
        return None

In [11]:
cup_table_tea_oz["amount_oz"] = cup_table_tea_oz.apply(
    lambda row: convert_to_oz(row["amount"], row["unit"]), axis=1
)

In [12]:
cup_table_tea_oz

,amount,unit,amount_oz
0,4.00,tablespoon,2.000000
1,2.00,tablespoon,1.000000
2,1.00,tablespoon,0.500000
3,1.00,cup,8.000000
4,2.00,cup,16.000000
...,...,...,...
3688,0.50,teaspoon,0.083350
3693,2.00,tablespoon,1.000000
3694,0.50,teaspoon,0.083350
3699,0.25,teaspoon,0.041675


In [13]:
all_units = ingre_units['unit'].dropna().str.lower().str.strip().unique()

In [14]:
# now lets explore the other things

all_units = ingre_units['unit'].dropna().str.lower().str.strip().unique()
extra_units = [u for u in all_units if u not in unit_to_oz]

extra_counts = (
    ingre_units[ingre_units['unit'].isin(extra_units)]
    ['unit']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'unit', 'unit': 'count'})
)
extra_counts

,count,count
0,package,207
1,bag,144
2,packages,57
3,slices,52
4,can,47
...,...,...
87,large tub,1
88,bar * ounce,1
89,bunches,1
90,stems,1


In [15]:
# this allows us to explore the extra_counts and see if they can match up

def explore_ingredients(unit_name):
    unit_df = cookbook[cookbook['unit'] == unit_name].copy()
    unit_reduced = unit_df[['name', 'amount', 'unit']]
    tj_reduced = tj_inventory[['product_name', 'store_amount', 'store_unit']]
    matches = unit_reduced.merge(tj_reduced, left_on='name', right_on='product_name', how='inner')
    matches['amount'] = pd.to_numeric(matches['amount'], errors='coerce')
    matches['store_amount'] = pd.to_numeric(matches['store_amount'], errors='coerce')
    matches['amount_to_store_amount'] = matches['amount'] * matches['store_amount']
    matches_reduced = matches[['name', 'amount', 'unit','store_amount','amount_to_store_amount', 'store_unit']]
    return matches_reduced

explore_ingredients("pint")

,name,amount,unit,store_amount,amount_to_store_amount,store_unit


In [16]:
# does not work for tablespoon, teaspoon, slices, loaf, cloves, pound, stick, sprig, slice, scoops, bar, pint, heads and the rest under 3 count

In [17]:
# first attempt at merging convert_to_oz and explore_ingredients into one function
def units_to_store_units(df, tj_inventory):
    final_df = df.copy()

    first_units = [
        "cup", "cups",
        "tablespoon", "tablespoons",
        "teaspoon", "teaspoons",
        "ounce", "ounces"
    ]

    does_not_work = [
        "slices", "slice",
        "clove", "cloves",
        "handful", "sprig", "sprigs",
        'loaf', 'pound', 'scoops', 'bar', 'pint',
        'heads'
    ]

    unit_to_oz = {
        "cup": 8, "cups": 8,
        "tablespoon": 0.5, "tablespoons": 0.5,
        "teaspoon": 0.1667, "teaspoons": 0.1667,
        "ounce": 1, "ounces": 1
    }

    def convert_to_oz(amount, unit):
        """Converts a recipe's quantity + unit to ounces."""
        return float(amount) * unit_to_oz[unit]


    def explore_ingredients(unit_name):
        """Merges cookbook ingredients with TJ inventory by name."""
        unit_df = df[df["unit"] == unit_name].copy()
        unit_reduced = unit_df[["name", "amount", "unit"]]
        tj_reduced = tj_inventory[["product_name", "store_amount", "store_unit"]]
        matches = unit_reduced.merge(
            tj_reduced,
            left_on="name",
            right_on="product_name",
            how="inner"
        )
        matches["amount"] = pd.to_numeric(matches["amount"], errors="coerce")
        matches["store_amount"] = pd.to_numeric(matches["store_amount"], errors="coerce")
        matches["amount_to_store"] = matches["amount"] * matches["store_amount"]
        return matches[["name", "amount_to_store", "store_unit"]]

    # handle first_units (direct conversion to Oz)
    mask_first = final_df["unit"].isin(first_units)
    final_df.loc[mask_first, "store_amount"] = final_df.loc[mask_first].apply(
        lambda r: convert_to_oz(r["amount"], r["unit"]), axis=1
    )
    final_df.loc[mask_first, "store_unit"] = "Oz"

    # skip does_not_work
    mask_skip = final_df["unit"].isin(does_not_work)
    final_df.loc[mask_skip, ["store_amount", "store_unit"]] = [None, None]

    # everything else
    mask_other = ~(mask_first | mask_skip)
    unique_units = final_df.loc[mask_other, "unit"].unique()

    for unit_name in unique_units:
        matches = explore_ingredients(unit_name)
        if not matches.empty:
            for _, row in matches.iterrows():
                final_df.loc[
                    (final_df["name"] == row["name"]) & (final_df["unit"] == unit_name),
                    ["store_amount", "store_unit"]
                ] = [row["amount_to_store"], row["store_unit"]]

    return final_df

In [18]:
# final attempt. this takes the df, converts first units with convert_to_oz, then where it can convert using explore_ingredients, if it can't do either it brings over 'amount' and 'unit'
def units_to_store_units2(df, tj_inventory):
    final_df = df.copy()

    first_units = [
        "cup", "cups",
        "tablespoon", "tablespoons",
        "teaspoon", "teaspoons",
        "ounce", "ounces"
    ]

    does_not_work = [
        # "nothing"
        "slices", "slice",
        "clove", "cloves",
        "handful", "sprig", "sprigs",
        "loaf", "pound", "scoops", "bar", "pint",
        "heads"
    ]

    unit_to_oz = {
        "cup": 8, "cups": 8,
        "tablespoon": 0.5, "tablespoons": 0.5,
        "teaspoon": 0.1667, "teaspoons": 0.1667,
        "ounce": 1, "ounces": 1
    }

    def convert_to_oz(amount, unit):
        """Converts a recipe's quantity + unit to ounces."""
        try:
            return float(amount) * unit_to_oz[unit]
        except (TypeError, ValueError, KeyError):
            return None

    def explore_ingredients(unit_name):
        """Merges cookbook ingredients with TJ inventory by name."""
        unit_df = df[df["unit"] == unit_name].copy()
        unit_reduced = unit_df[["name", "amount", "unit"]]
        tj_reduced = tj_inventory[["product_name", "store_amount", "store_unit"]]
        matches = unit_reduced.merge(
            tj_reduced,
            left_on="name",
            right_on="product_name",
            how="inner"
        )
        matches["amount"] = pd.to_numeric(matches["amount"], errors="coerce")
        matches["store_amount"] = pd.to_numeric(matches["store_amount"], errors="coerce")
        matches["amount_to_store"] = matches["amount"] * matches["store_amount"]
        matches.rename(
            columns={
                "amount_to_store": "pantry_amount",
                "store_unit": "pantry_unit"
            },
            inplace=True
        )
        return matches[["name", "pantry_amount", "pantry_unit"]]

    # ---- 1) handle first_units (direct conversion to Oz) ----
    mask_first = final_df["unit"].isin(first_units)
    final_df.loc[mask_first, "pantry_amount"] = final_df.loc[mask_first].apply(
        lambda r: convert_to_oz(r["amount"], r["unit"]), axis=1
    )
    final_df.loc[mask_first, "pantry_unit"] = "Oz"

    # ---- 2) skip does_not_work ----
    mask_skip = final_df["unit"].isin(does_not_work)
    final_df.loc[mask_skip, ["pantry_amount", "pantry_unit"]] = [None, None]

    # ---- 3) everything else ----
    mask_other = ~(mask_first | mask_skip)
    unique_units = final_df.loc[mask_other, "unit"].unique()

    for unit_name in unique_units:
        matches = explore_ingredients(unit_name)
        if not matches.empty:
            for _, row in matches.iterrows():
                final_df.loc[
                    (final_df["name"] == row["name"]) & (final_df["unit"] == unit_name),
                    ["pantry_amount", "pantry_unit"]
                ] = [row["pantry_amount"], row["pantry_unit"]]

    # ---- 4) mark converted_to_store ----
    final_df["converted_to_store"] = (
        final_df["pantry_amount"].notna() & final_df["pantry_unit"].notna()
    )

    # ---- 5) fallback: if not converted, copy original unit + amount ----
    not_converted = ~final_df["converted_to_store"]
    final_df.loc[not_converted, "pantry_amount"] = final_df.loc[not_converted, "amount"]
    final_df.loc[not_converted, "pantry_unit"] = final_df.loc[not_converted, "unit"]

    return final_df

In [19]:
# dropping na since we can't convert if we don't have units and amounts
cookbook2 = cookbook.dropna().copy()

In [20]:
cookbook_revised2 = units_to_store_units2(cookbook2, tj_inventory)
cookbook_revised2

,original_text,name,name_confidence,amount,amount_text,unit,amount_confidence,preparation,preparation_confidence,recipe_title,...,likely_sub_category_3,likely_sub_category_3_score,main_category_3,matched_products,units,matched_categories,match_scores,pantry_amount,pantry_unit,converted_to_store
1,2 tablespoons finely chopped TJ’s Fresh Garlic,Fresh Garlic,0.942706,2.0,2 tablespoons,tablespoon,0.999844,finely chopped,0.961860,Aromatic Garlic Ginger Rice,...,Spices,0.037934,For the Pantry,Garlic & Jalapeño Stuffed Olives,17.64 Oz,For the Pantry,0.735,1.0000,Oz,True
2,1 tablespoon finely chopped TJ’s Fresh Ginger,Fresh Ginger,0.992840,1.0,1 tablespoon,tablespoon,0.999925,finely chopped,0.997657,Aromatic Garlic Ginger Rice,...,Fruits,0.010147,Fresh Fruits & Veggies,Miso Ginger Broth,32 Fl Oz,For the Pantry,0.724,0.5000,Oz,True
3,"1 cup TJ’s Jasmine Rice, rinsed",Jasmine Rice,0.997608,1.0,1 cup,cup,0.999978,rinsed,0.998351,Aromatic Garlic Ginger Rice,...,Spices,0.051027,For the Pantry,Jasmine Rice From Thailand,3 Lb,For the Pantry,0.951,8.0000,Oz,True
9,"3 tablespoons TJ’s Avocado Oil, divided",Avocado Oil,0.997538,3.0,3 tablespoons,tablespoon,0.999973,divided,0.999657,Roasted Pork Tenderloin & Potatoes with Honey ...,...,Cool Desserts,0.068727,From The Freezer,Avocado Oil; Avocado Spray Oil,16.9 Fl Oz; 5 Oz,For the Pantry; For the Pantry,0.843; 0.777,1.5000,Oz,True
10,"1 bag TJ’s Teeny Tiny Potatoes, sliced in half...",Teeny Tiny Potatoes,0.994637,1.0,1 bag,bag,0.999593,"sliced in half, lengthwise",0.928255,Roasted Pork Tenderloin & Potatoes with Honey ...,...,Cereals,0.083543,For the Pantry,Organic Petite Potato Medley,3.53 Oz; 6 Oz,Fresh Fruits & Veggies; Snacks & Sweets,0.724; 0.708,1.0000,bag,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,4 teaspoons TJ's Everything But The Leftovers ...,Everything But The Leftovers Seasoning,0.894283,4.0,4 teaspoons,teaspoon,0.999109,divided,0.997654,Everything but the Leftovers Sheet Pan Dinner,...,Spices,0.122283,For the Pantry,Everything But The Leftovers Seasoning Blend; ...,2.6 Oz; 2.3 Oz; 2.3 Oz,Spices; For the Pantry; For the Pantry,0.759; 0.724; 0.716,0.6668,Oz,True
3682,1 1/2 cups TJ's Organic Foursome Vegetable Med...,Organic Foursome Vegetable Medley,0.998896,1.5,1 1/2 cups,cup,0.999944,thawed,0.964607,Turkey Pot Pie with Buttermilk Biscuit Crust,...,Cereals,0.039006,For the Pantry,Organic Petite Potato Medley,24 Oz,Fresh Fruits & Veggies,0.7,12.0000,Oz,True
3685,2 cups chopped leftover cooked TJ's Turkey,leftover cooked Turkey,0.890696,2.0,2 cups,cup,0.999918,chopped,0.985658,Turkey Pot Pie with Buttermilk Biscuit Crust,...,"Beef, Pork & Lamb",0.037698,"Meat, Seafood & Plant-based",Grilled Chili Lime Chicken Breast; Chicken Mea...,12 Oz; 12 Oz; 6 Oz,"Meat, Seafood & Plant-based; Meat, Seafood & P...",0.716; 0.725; 0.708,16.0000,Oz,True
3686,2 teaspoons chopped TJ's Fresh Thyme,Fresh Thyme,0.951175,2.0,2 teaspoons,teaspoon,0.999752,chopped,0.958873,Turkey Pot Pie with Buttermilk Biscuit Crust,...,Dessert & Sweets,0.005228,Fresh Prepared Foods,Chicken Cilantro Mini Wontons; Organic Cranber...,12 Oz; 12 Oz; 32 Fl Oz,From The Freezer; Fresh Fruits & Veggies; For ...,0.724; 0.836; 0.732,0.3334,Oz,True


In [21]:
# cookbook2.to_csv("../quantity_exploration/parsed_ingredients_with_pantry_units.csv", index=False)

In [22]:
# the following was used to explore the first iteration of units_to_store_units
unit_nan_pct = cookbook_revised2['unit'].isna().mean() * 100
store_unit_nan_pct = cookbook_revised2['pantry_unit'].isna().mean() * 100

print(f"NaN % in unit: {unit_nan_pct:.1f}%")
print(f"NaN % in store_unit: {store_unit_nan_pct:.1f}%")
print(f"Increase: {store_unit_nan_pct - unit_nan_pct:.1f}%")

NaN % in unit: 0.0%
NaN % in store_unit: 0.0%
Increase: 0.0%


In [23]:
# Find rows that used to have a unit but now lost store_unit
missing_after = cookbook_revised2[cookbook_revised2['unit'].notna() & cookbook_revised2['pantry_unit'].isna()]
print(f"{len(missing_after)} rows lost store_unit after conversion.")
missing_after

0 rows lost store_unit after conversion.


,original_text,name,name_confidence,amount,amount_text,unit,amount_confidence,preparation,preparation_confidence,recipe_title,...,likely_sub_category_3,likely_sub_category_3_score,main_category_3,matched_products,units,matched_categories,match_scores,pantry_amount,pantry_unit,converted_to_store
